In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')
sns.set(font_scale=1.2)
sns.set_theme(style="whitegrid")

DATA_DIR = '../data' #You many change the directory

In [4]:
users = pd.read_csv('{}/users.csv'.format(DATA_DIR))
events = pd.read_csv('{}/events.csv'.format(DATA_DIR))
transactions = pd.read_csv('{}/transactions.csv'.format(DATA_DIR))
sessions = pd.read_csv('{}/learn_sessions_transactions.csv.gz'.format(DATA_DIR))
topics = pd.read_csv('{}/topics_translated.csv'.format(DATA_DIR))
documents = pd.read_csv('{}/documents.csv'.format(DATA_DIR))

# 1. Merge tables

In [5]:
def remove_users(events, transactions):
    tokens_event = set(events['transaction_token'].unique())
    print("{} tokens in events".format(len(tokens_event)))

    tokens_trans = set(transactions['transaction_token'].unique())
    print("{} tokens in transactions".format(len(tokens_trans)))

    missing_tokens = list(tokens_event - tokens_trans)
    missing_tokens.remove(np.nan)
    print("{} tokens in events but not in transactions".format(len(missing_tokens)))

    users_missing = list(events[events['transaction_token'].isin(missing_tokens)]['user_id'].unique())
    print("{} users in events but not in transactions".format(len(users_missing)))

    # Events without missing users. Equivalent to removing the missing tokens from events.
    len_events = len(events)
    events = events[~events['user_id'].isin(users_missing)]
    stat_events = (len_events - len(events))/len_events
    print("{}% of events eliminated".format(round(stat_events, 4)))

    return events, transactions

events, transactions = remove_users(events, transactions)

637678 tokens in events
800018 tokens in transactions
37685 tokens in events but not in transactions
1304 users in events but not in transactions
0.0595% of events eliminated


In [6]:
df = pd.merge(events, users, on='user_id')
df = df.merge(transactions, on=['user_id', 'transaction_token'], how='left')
df = df.merge(sessions, on='transaction_id', how='left')
df['evaluation'] = df['evaluation'].map({"CORRECT": 1, "PARTIAL": 0.5, "WRONG": 0})
df = df.drop(['transaction_token', 'learn_session_id', 'max_num_tasks', 'user_agent'], axis=1)
df

,event_id,user_id,timestamp,category,action,event_type,session_id,tracking_data,gender,canton,...,input,start_time,commit_time,validation,solution,type,topic_id,is_closed,type_id,is_accepted
0,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN
1,70,392476,1621584365975,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN
2,72,392476,1621584441478,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN
3,73,392476,1621584446771,TASK,GO_TO_THEORY,CLICK,33768.0,NaN,NaN,NaN,...,NaN,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN
4,54076,392476,1625054796765,GENERAL,LOGIN,CLICK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259194,3516476,389181,1645514034678,DASHBOARD,NAVIGATE_DASHBOARD,VIEW,NaN,"{""dashboard"": {""title"": ""Mathematik"", ""topics""...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259195,3516528,389181,1645514507946,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259196,3516529,389181,1645514507951,USER_SESSION,WINDOW_VISIBLE_TRUE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259197,3516530,389181,1645514523129,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import json
def get_difficulty(doc):
    res = None
    doc = json.loads(doc)
    if doc.get('metaData') is not None:
        res = doc.get('metaData').get('estimatedDifficulty')
    return res

documents.drop_duplicates(['document_id'], keep='last', inplace=True)
documents['difficulty'] = documents['content'].apply(lambda x: get_difficulty(x))
df = df.merge(documents[['document_id', 'difficulty']], on='document_id', how='left')
df

,event_id,user_id,timestamp,category,action,event_type,session_id,tracking_data,gender,canton,...,start_time,commit_time,validation,solution,type,topic_id,is_closed,type_id,is_accepted,difficulty
0,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0
1,70,392476,1621584365975,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0
2,72,392476,1621584441478,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0
3,73,392476,1621584446771,TASK,GO_TO_THEORY,CLICK,33768.0,NaN,NaN,NaN,...,1.620304e+12,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0
4,54076,392476,1625054796765,GENERAL,LOGIN,CLICK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259194,3516476,389181,1645514034678,DASHBOARD,NAVIGATE_DASHBOARD,VIEW,NaN,"{""dashboard"": {""title"": ""Mathematik"", ""topics""...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259195,3516528,389181,1645514507946,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259196,3516529,389181,1645514507951,USER_SESSION,WINDOW_VISIBLE_TRUE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259197,3516530,389181,1645514523129,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
topics.rename(columns={'id': 'topic_id'}, inplace=True)
topics['german'] = abs(1 - topics['math'])
df = df.merge(topics[['topic_id', 'math', 'german']], on='topic_id', how='left')
df

,event_id,user_id,timestamp,category,action,event_type,session_id,tracking_data,gender,canton,...,validation,solution,type,topic_id,is_closed,type_id,is_accepted,difficulty,math,german
0,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0,0.0
1,70,392476,1621584365975,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0,0.0
2,72,392476,1621584441478,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0,0.0
3,73,392476,1621584446771,TASK,GO_TO_THEORY,CLICK,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0,0.0
4,54076,392476,1625054796765,GENERAL,LOGIN,CLICK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259194,3516476,389181,1645514034678,DASHBOARD,NAVIGATE_DASHBOARD,VIEW,NaN,"{""dashboard"": {""title"": ""Mathematik"", ""topics""...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259195,3516528,389181,1645514507946,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259196,3516529,389181,1645514507951,USER_SESSION,WINDOW_VISIBLE_TRUE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259197,3516530,389181,1645514523129,USER_SESSION,WINDOW_VISIBLE_FALSE,VIEW,NaN,"{""location"": ""https://app.lernnavi.ch/dashboar...",MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2.preprocess

In [9]:
df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date  # convert timestamp to pd.datetime
users_day_event = df.groupby(['user_id', 'date'])  # groupby users' daily

# number of questions user answers in a day
user_num_questions = users_day_event.apply(lambda x: len(x[x['action'] == 'SUBMIT_ANSWER']))
user_num_questions = user_num_questions.reset_index(name='num_questions')

user_stats = pd.merge(users, user_num_questions, on='user_id')
user_stats

,user_id,gender,canton,class_level,date,num_questions
0,387604,NaN,NaN,NaN,2021-05-22,0
1,387604,NaN,NaN,NaN,2021-05-28,0
2,387604,NaN,NaN,NaN,2021-06-23,0
3,387604,NaN,NaN,NaN,2021-07-02,0
4,387604,NaN,NaN,NaN,2021-08-06,0
...,...,...,...,...,...,...
38048,404599,NaN,NaN,NaN,2022-02-21,1
38049,404603,FEMALE,SO,Gymnasium - 4. Jahr,2022-02-21,13
38050,404604,FEMALE,ZH,Langzeitgymnasium - 2. Jahr,2022-02-21,4
38051,404605,FEMALE,SO,Gymnasium - 3. Jahr,2022-02-21,0


In [10]:
# number of users' action in a day
user_stats = user_stats.merge(users_day_event.apply(len).reset_index(name='num_events'), on=['user_id', 'date'])
user_stats

,user_id,gender,canton,class_level,date,num_questions,num_events
0,387604,NaN,NaN,NaN,2021-05-22,0,1
1,387604,NaN,NaN,NaN,2021-05-28,0,1
2,387604,NaN,NaN,NaN,2021-06-23,0,7
3,387604,NaN,NaN,NaN,2021-07-02,0,25
4,387604,NaN,NaN,NaN,2021-08-06,0,12
...,...,...,...,...,...,...,...
38048,404599,NaN,NaN,NaN,2022-02-21,1,169
38049,404603,FEMALE,SO,Gymnasium - 4. Jahr,2022-02-21,13,112
38050,404604,FEMALE,ZH,Langzeitgymnasium - 2. Jahr,2022-02-21,4,43
38051,404605,FEMALE,SO,Gymnasium - 3. Jahr,2022-02-21,0,22


In [19]:
# the percentage of correct answers in a day
percentage_correct = users_day_event.apply(lambda x: x['evaluation'].sum() / len(x[x['evaluation'] != np.nan]))
user_stats = user_stats.merge(percentage_correct.reset_index(name='percentage_correct'), on=['user_id', 'date'])
user_stats

,user_id,gender,canton,class_level,date,num_questions,num_events,percentage_correct
0,387604,NaN,NaN,NaN,2021-05-22,0,1,0.000000
1,387604,NaN,NaN,NaN,2021-05-28,0,1,0.000000
2,387604,NaN,NaN,NaN,2021-06-23,0,7,0.000000
3,387604,NaN,NaN,NaN,2021-07-02,0,25,0.000000
4,387604,NaN,NaN,NaN,2021-08-06,0,12,0.000000
...,...,...,...,...,...,...,...,...
38511,404564,MALE,SO,Gymnasium - 2. Jahr,2022-02-22,0,28,0.000000
38512,404565,MALE,SO,Gymnasium - 2. Jahr,2022-02-21,0,9,0.000000
38513,404565,MALE,SO,Gymnasium - 2. Jahr,2022-02-22,3,70,0.085714
38514,404598,MALE,SG,Gymnasium - 1. Jahr,2022-02-21,1,220,0.013636


In [33]:
duration = pd.to_timedelta(pd.to_datetime(users_day_event['timestamp'].max(), unit='ms')
                           - pd.to_datetime(users_day_event['timestamp'].min(), unit='ms')).dt.total_seconds() / 60
print(duration)

user_id  date      
387604   2021-05-22      0.000000
         2021-05-28      0.000000
         2021-06-23      0.685000
         2021-07-02     93.828200
         2021-08-06      1.367617
                          ...    
404600   2022-02-21     34.882900
404603   2022-02-21    311.390483
404604   2022-02-21    107.607033
404605   2022-02-21     45.804750
404623   2022-02-21      9.882383
Name: timestamp, Length: 41654, dtype: float64


In [12]:
user_stats = user_stats.groupby(['user_id']).filter(lambda x: len(x) > 1)  # filter out users only activate in one day
user_stats

,user_id,gender,canton,class_level,date,num_questions,num_events
0,387604,NaN,NaN,NaN,2021-05-22,0,1
1,387604,NaN,NaN,NaN,2021-05-28,0,1
2,387604,NaN,NaN,NaN,2021-06-23,0,7
3,387604,NaN,NaN,NaN,2021-07-02,0,25
4,387604,NaN,NaN,NaN,2021-08-06,0,12
...,...,...,...,...,...,...,...
38025,404564,MALE,SO,Gymnasium - 2. Jahr,2022-02-22,0,28
38026,404565,MALE,SO,Gymnasium - 2. Jahr,2022-02-21,0,9
38027,404565,MALE,SO,Gymnasium - 2. Jahr,2022-02-22,3,70
38046,404598,MALE,SG,Gymnasium - 1. Jahr,2022-02-21,1,220


In [15]:
session_length = users_day_event['session_id'].count()
print(session_length)

user_id  date      
387604   2021-05-22     0
         2021-05-28     0
         2021-06-23     3
         2021-07-02    17
         2021-08-06     0
                       ..
404600   2022-02-21     0
404603   2022-02-21    65
404604   2022-02-21    25
404605   2022-02-21     0
404623   2022-02-21     0
Name: session_id, Length: 41654, dtype: int64


In [39]:
users_day_event = users_day_event.merge(users, on='user_id', how='left')
users_day_event

,user_id,date,num_events,gender,canton,class_level
0,387604,2021-05-22,1,NaN,NaN,NaN
1,387604,2021-05-28,1,NaN,NaN,NaN
2,387604,2021-06-23,7,NaN,NaN,NaN
3,387604,2021-07-02,25,NaN,NaN,NaN
4,387604,2021-08-06,12,NaN,NaN,NaN
...,...,...,...,...,...,...
38048,404599,2022-02-21,169,NaN,NaN,NaN
38049,404603,2022-02-21,112,FEMALE,SO,Gymnasium - 4. Jahr
38050,404604,2022-02-21,43,FEMALE,ZH,Langzeitgymnasium - 2. Jahr
38051,404605,2022-02-21,22,FEMALE,SO,Gymnasium - 3. Jahr


,user_id,date,num_events,gender,canton,class_level
0,387604,2021-05-22,1,NaN,NaN,NaN
1,387604,2021-05-28,1,NaN,NaN,NaN
2,387604,2021-06-23,7,NaN,NaN,NaN
3,387604,2021-07-02,25,NaN,NaN,NaN
4,387604,2021-08-06,12,NaN,NaN,NaN
...,...,...,...,...,...,...
38025,404564,2022-02-22,28,MALE,SO,Gymnasium - 2. Jahr
38026,404565,2022-02-21,9,MALE,SO,Gymnasium - 2. Jahr
38027,404565,2022-02-22,70,MALE,SO,Gymnasium - 2. Jahr
38046,404598,2022-02-21,220,MALE,SG,Gymnasium - 1. Jahr


In [13]:
from datetime import timedelta
user_stats['date'].quantile(0.8) + timedelta(days=7)

datetime.date(2022, 1, 26)

In [15]:
user_stats['date_shift'] = user_stats.groupby(['user_id'])['date'].shift(-1)
user_stats_labeled = user_stats.dropna(subset=['date_shift'])
user_stats_labeled['label'] = user_stats['date'] + timedelta(days=7) >= user_stats['date_shift']
user_stats_labeled

,user_id,gender,canton,class_level,date,num_questions,num_events,date_shift,label
0,387604,NaN,NaN,NaN,2021-05-22,0,1,2021-05-28,True
1,387604,NaN,NaN,NaN,2021-05-28,0,1,2021-06-23,False
2,387604,NaN,NaN,NaN,2021-06-23,0,7,2021-07-02,False
3,387604,NaN,NaN,NaN,2021-07-02,0,25,2021-08-06,False
4,387604,NaN,NaN,NaN,2021-08-06,0,12,2021-08-13,True
...,...,...,...,...,...,...,...,...,...
38020,404561,FEMALE,SO,Gymnasium - 2. Jahr,2022-02-21,1,48,2022-02-22,True
38022,404563,FEMALE,SO,Gymnasium - 2. Jahr,2022-02-21,0,69,2022-02-22,True
38024,404564,MALE,SO,Gymnasium - 2. Jahr,2022-02-21,1,32,2022-02-22,True
38026,404565,MALE,SO,Gymnasium - 2. Jahr,2022-02-21,0,9,2022-02-22,True


In [44]:
group_stats = user_stats_labeled.groupby('user_id')
len(group_stats)

6150

In [45]:
from collections import defaultdict
dates = sorted(user_stats_labeled['date'].unique())

datas = defaultdict(list)
gp_sub = []
num_data = 0
for date in dates:
    print(date)
    sub_data = user_stats_labeled[user_stats_labeled['date'] == date]
    sub_data = user_stats_labeled[user_stats_labeled['user_id'].apply(lambda x: x in sub_data['user_id'].values)]
    sub_data = sub_data[sub_data['date'] <= date]
    group_sub_data = sub_data.groupby('user_id')
    for window_size in range(1, group_sub_data.apply(len).max()+1):
        group_sub_data = group_sub_data.filter(lambda x: len(x) >= window_size).groupby('user_id')
        num_data += len(group_sub_data)
        slice_data = group_sub_data.tail(window_size).reset_index(drop=True)
        datas[window_size].append(slice_data)

    # tmp = sub_data.drop_duplicates('user_id', keep='last')
    # tmp['activity_day'] = sub_data.groupby('user_id').apply(len).values
    # gp_sub.append(sub_data.groupby('user_id').apply(len))
    # datas.append(tmp)


2021-05-21
2021-05-22
2021-05-23
2021-05-24
2021-05-25
2021-05-26
2021-05-27
2021-05-28
2021-05-29
2021-05-30
2021-05-31
2021-06-01
2021-06-02
2021-06-03
2021-06-04
2021-06-05
2021-06-06
2021-06-07
2021-06-08
2021-06-09
2021-06-10
2021-06-11
2021-06-12
2021-06-13
2021-06-14
2021-06-15
2021-06-16
2021-06-17
2021-06-18
2021-06-19
2021-06-20
2021-06-21
2021-06-22
2021-06-23
2021-06-24
2021-06-25
2021-06-26
2021-06-27
2021-06-28
2021-06-29
2021-06-30
2021-07-01
2021-07-02
2021-07-03
2021-07-04
2021-07-05
2021-07-06
2021-07-07
2021-07-08
2021-07-09
2021-07-10
2021-07-11
2021-07-12
2021-07-13
2021-07-14
2021-07-15
2021-07-16
2021-07-17
2021-07-18
2021-07-19
2021-07-20
2021-07-21
2021-07-22
2021-07-23
2021-07-24
2021-07-25
2021-07-26
2021-07-27
2021-07-28
2021-07-29
2021-07-30
2021-07-31
2021-08-01
2021-08-03
2021-08-04
2021-08-05
2021-08-06
2021-08-07
2021-08-08
2021-08-09
2021-08-10
2021-08-11
2021-08-12
2021-08-13
2021-08-14
2021-08-15
2021-08-16
2021-08-17
2021-08-18
2021-08-19
2021-08-20

In [46]:
# datas[2][1]
num_data

207821

In [39]:
num = 0
for x in datas.values():
    num+=len(x)
num

12478

In [33]:
date = pd.to_datetime('2021-08-02')
sub_data = user_stats_labeled[user_stats_labeled['date'] ==date]
sub_data = user_stats_labeled[user_stats_labeled['user_id'].apply(lambda x: x in sub_data['user_id'].values)]
sub_data = sub_data[sub_data['date'] <= date]
group_sub_data = sub_data.groupby('user_id')
# sub_data
user_stats_labeled[user_stats_labeled['date'] ==date]

,user_id,gender,canton,class_level,date,num_questions,num_events,date_shift,label


Timestamp('2021-08-02 00:00:00')

In [76]:
users_day_event_labeled[users_day_event_labeled['user_id'] == 393262]

,user_id,date,num_events,gender,canton,class_level,date_shift,label
5841,393262,2021-05-26,19,FEMALE,SG,FMS - 2. Jahr,2021-05-28,True
5842,393262,2021-05-28,10,FEMALE,SG,FMS - 2. Jahr,2021-05-30,True
5843,393262,2021-05-30,1,FEMALE,SG,FMS - 2. Jahr,2021-05-31,True
5844,393262,2021-05-31,6,FEMALE,SG,FMS - 2. Jahr,2022-01-03,False
5845,393262,2022-01-03,32,FEMALE,SG,FMS - 2. Jahr,2022-01-07,True
5846,393262,2022-01-07,77,FEMALE,SG,FMS - 2. Jahr,2022-01-14,True
5847,393262,2022-01-14,38,FEMALE,SG,FMS - 2. Jahr,2022-01-28,False
5848,393262,2022-01-28,45,FEMALE,SG,FMS - 2. Jahr,2022-02-17,False
5849,393262,2022-02-17,15,FEMALE,SG,FMS - 2. Jahr,2022-02-18,True


In [23]:
topics.rename(columns={'id': 'topic_id'}, inplace=True)

,event_id,user_id,timestamp,category,action,event_type,session_id,tracking_data,gender,canton,...,topic_id,is_closed,type_id,is_accepted,difficulty,name,description,name_english,description_english,math
0,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,951.0,0.0,1.0,NaN,NaN,Zahlenmengen,Die Schülerinnen und Schüler kennen die Zahlen...,Quantities,The students know the number of numbers and th...,1.0
1,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,951.0,0.0,1.0,NaN,NaN,Zahlenmengen,Die Schülerinnen und Schüler kennen die Zahlen...,Quantities,The students know the number of numbers and th...,1.0
2,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,951.0,0.0,1.0,NaN,NaN,Zahlenmengen,Die Schülerinnen und Schüler kennen die Zahlen...,Quantities,The students know the number of numbers and th...,1.0
3,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,951.0,0.0,1.0,NaN,1.0,Zahlenmengen,Die Schülerinnen und Schüler kennen die Zahlen...,Quantities,The students know the number of numbers and th...,1.0
4,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,951.0,0.0,1.0,NaN,1.0,Zahlenmengen,Die Schülerinnen und Schüler kennen die Zahlen...,Quantities,The students know the number of numbers and th...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13095968,3516474,389181,1645514034057,SESSION,CLOSE,CLICK,112223.0,NaN,MALE,SG,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
13095969,3516474,389181,1645514034057,SESSION,CLOSE,CLICK,112223.0,NaN,MALE,SG,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
13095970,3516474,389181,1645514034057,SESSION,CLOSE,CLICK,112223.0,NaN,MALE,SG,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
13095971,3516474,389181,1645514034057,SESSION,CLOSE,CLICK,112223.0,NaN,MALE,SG,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [31]:
df2 = df2.merge(topics[['topic_id', 'math']], on='topic_id', how='left')
df2

,event_id,user_id,timestamp,category,action,event_type,session_id,tracking_data,gender,canton,...,commit_time,validation,solution,type,topic_id,is_closed,type_id,is_accepted,difficulty,math
0,69,392476,1621584325174,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0
1,70,392476,1621584365975,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0
2,72,392476,1621584441478,TASK,VIEW_QUESTION,VIEW,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0
3,73,392476,1621584446771,TASK,GO_TO_THEORY,CLICK,33768.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,951.0,0.0,1.0,NaN,2.0,1.0
4,223,393213,1621589245107,TASK,VIEW_QUESTION,VIEW,34343.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,952.0,0.0,1.0,NaN,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506881,3516466,389181,1645514014647,FEEDBACK,OPEN_FEEDBACK,CLICK,112223.0,"{""popoverId"": ""e6b42e8e-542e-4ba0-a6c0-dc5f498...",MALE,SG,...,1.645514e+12,PASS,"{""gaps"": [""Richtig"", ""Nicht notwendig"", ""Richt...",CLOZE_TEXT_DROPDOWN,978.0,1.0,1.0,1.0,1.0,1.0
1506882,3516467,389181,1645514016163,FEEDBACK,CLOSE_FEEDBACK,CLICK,112223.0,"{""popoverId"": ""e6b42e8e-542e-4ba0-a6c0-dc5f498...",MALE,SG,...,1.645514e+12,PASS,"{""gaps"": [""Richtig"", ""Nicht notwendig"", ""Richt...",CLOZE_TEXT_DROPDOWN,978.0,1.0,1.0,1.0,1.0,1.0
1506883,3516468,389181,1645514024160,TASK,NEXT,CLICK,112223.0,NaN,MALE,SG,...,1.645514e+12,PASS,"{""gaps"": [""Richtig"", ""Nicht notwendig"", ""Richt...",CLOZE_TEXT_DROPDOWN,978.0,1.0,1.0,1.0,1.0,1.0
1506884,3516473,389181,1645514033264,TASK,VIEW_QUESTION,VIEW,112223.0,NaN,MALE,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [22]:
df_group = df2.groupby('user_id')

In [31]:
def count_navi_num(x):
    if 'NAVIGATE_DASHBOARD' in x['action'].values:
        return x['action'].value_counts()['NAVIGATE_DASHBOARD']
    else:
        return 0
df_group.apply(count_navi_num).sort_values(ascending=False)

user_id
396334    971
393686    864
394739    847
394987    841
393959    821
         ... 
400727      0
403229      0
403230      0
400715      0
400180      0
Length: 10113, dtype: int64

In [33]:
def count_level_check_num(x):
    x = x.drop_duplicates('session_id')
    return len(x[x['type_id'] == 2])
df_group.apply(count_level_check_num).sort_values(ascending=False)

user_id
396341    79
401331    61
393959    58
394171    52
396334    52
          ..
398567     0
398566     0
398565     0
398564     0
404623     0
Length: 10113, dtype: int64

In [30]:
df2['action'].value_counts()['NAVIGATE_DASHBOARD']

350821

In [27]:
'NAVIGATE_DASHBOARD' in df2['action']

False

In [28]:
df2['action']

0                 VIEW_QUESTION
1                 SUBMIT_ANSWER
2                          NEXT
3                 VIEW_QUESTION
4                 SUBMIT_ANSWER
                   ...         
3465554      NAVIGATE_DASHBOARD
3465555    WINDOW_VISIBLE_FALSE
3465556     WINDOW_VISIBLE_TRUE
3465557    WINDOW_VISIBLE_FALSE
3465558            WINDOW_CLOSE
Name: action, Length: 3465559, dtype: object